## Example: using Linear Programming to solve for 3 latent factors `w`

Please note that the last part of this example, "Permutation invariance in linear programming vs. MultiCCA in R and `pmd`" requires that `inpt3` be exactly as printed.

In [34]:
import pandas as pd
import numpy as np

from sparsecca import lp_pmd

In [35]:
# example inputs
inpt1 = pd.read_csv("../tests/data/multicca1.csv", sep=",", index_col=0).values
inpt2 = pd.read_csv("../tests/data/multicca2.csv", sep=",", index_col=0).values[:, -5:]
inpt3 = np.random.normal(size=inpt2.shape)

penalties = [1.5, 1.5, 1.5]

In [36]:
weights, svd_init = lp_pmd(
    datasets=[inpt1, inpt2], # match feature dimension for now
    penalties=penalties[:2],
    K=3,
    standardize=True,
    mimic_R=True
)

## Correlation between IO results and LP results

In [37]:
from scipy.stats import spearmanr
from sparsecca import multicca_pmd

### high correlation between pmd and lp when `n=2`

Let `n` equal the number of datasets.

In [38]:
datasets = [inpt1, inpt2]
ws_lp, _ = lp_pmd(datasets, penalties[:2])
ws_io, _ = multicca_pmd(datasets, penalties[:2])

In [39]:
print(spearmanr(ws_lp[0], ws_io[0]))
print(spearmanr(ws_lp[1], ws_io[1]))

SignificanceResult(statistic=0.9746794344808964, pvalue=0.004818230468198537)
SignificanceResult(statistic=0.8999999999999998, pvalue=0.03738607346849874)


Note that this is the same as taking the first latent factor when K=3, as per the implementation in Witten 2009.

In [40]:
print(spearmanr(weights[0].T[0], ws_io[0]))
print(spearmanr(weights[1].T[0], ws_io[1]))

SignificanceResult(statistic=0.9746794344808964, pvalue=0.004818230468198537)
SignificanceResult(statistic=0.8999999999999998, pvalue=0.03738607346849874)


### arbitrarily lower correlation when `n=3`

Compare when `n=3`. Note that this is subject to the same warning as in the bottom of the notebook.

In [41]:
datasets = [inpt1, inpt2, inpt3]
ws_lp, _ = lp_pmd(datasets, penalties)
ws_io, _ = multicca_pmd(datasets, penalties)

In [42]:
print(spearmanr(ws_lp[0], ws_io[0]))
print(spearmanr(ws_lp[1], ws_io[1]))
print(spearmanr(ws_lp[2], ws_io[2]))

SignificanceResult(statistic=0.20519567041703082, pvalue=0.7405819415910722)
SignificanceResult(statistic=-0.8999999999999998, pvalue=0.03738607346849874)
SignificanceResult(statistic=0.09999999999999999, pvalue=0.8728885715695383)


#### Optimization of the objective function

Size of the objective function, which was maximized:

In [43]:
print('1st pair:', ws_lp[0].T @ inpt1.T @ inpt2 @ ws_lp[1])
print('2nd pair:', ws_lp[1].T @ inpt2.T @ inpt3 @ ws_lp[2])
print('3rd pair:', ws_lp[2].T @ inpt3.T @ inpt1 @ ws_lp[0])
print('sum', ws_lp[0].T @ inpt1.T @ inpt2 @ ws_lp[1] + ws_lp[1].T @ inpt2.T @ inpt3 @ ws_lp[2] + ws_lp[2].T @ inpt3.T @ inpt1 @ ws_lp[0])

1st pair: [[119.0135171]]
2nd pair: [[34.50213156]]
3rd pair: [[30.66377104]]
sum [[184.1794197]]


In [44]:
print('1st pair:', ws_io[0].T @ inpt1.T @ inpt2 @ ws_io[1])
print('2nd pair:', ws_io[1].T @ inpt2.T @ inpt3 @ ws_io[2])
print('3rd pair:', ws_io[2].T @ inpt3.T @ inpt1 @ ws_io[0])
print('sum', ws_io[0].T @ inpt1.T @ inpt2 @ ws_io[1] + ws_io[1].T @ inpt2.T @ inpt3 @ ws_io[2] + ws_io[2].T @ inpt3.T @ inpt1 @ ws_io[0])

1st pair: [[56.9713026]]
2nd pair: [[18.63386997]]
3rd pair: [[29.98425782]]
sum [[105.58943039]]


Neither linear programming nor manual convergence consistently produces a better optimization of the objective function in this small test case.

L2 norm of the latent factors.

In [45]:
np.linalg.norm(ws_lp)

1.73205081608457

In [46]:
np.linalg.norm(ws_io)

1.7320508075688772

## Permutation invariance in linear programming vs. MultiCCA in R and `pmd`

In [47]:
def test_weights(weights_a, weights_b, perm_b: list[int], dec=5):
    """Tests whether `weights_a` and `weights_b` are the same given the permutation order of b.

    Parameters:
        weights_a: output of lp_pmd 
        weights_b: output of lp_pmd permuted Xn
                   -> weights are of type np.ndarray in shape (N, f, K)
                    - N: len(Xn) datasets
                    - f: amount of features
                    - K: amount of MCPs
        perm_b:    order of the datasets used to generate a, in b
        dec:       decimals to which weights should be rounded to account for numerical tolerance

    Returns:
        boolean: True if rounded weights are the same, else False
    """
    
    weights_a_rounded = np.array(weights_a).round(decimals=dec)
    weights_b_rounded = np.array(weights_b).round(decimals=dec)
    
    weights_b_ordered = []
    for o in perm_b:
        weights_b_ordered.append(weights_b_rounded[o])
        
    return all(x==True for x in (weights_a_rounded==weights_b_ordered).flatten())

In [48]:
datasets = [inpt1, inpt2, inpt3]
# original dataset with permutation 1, 2, 0
datasets_perm = [inpt3, inpt1, inpt2]

### Linear Programming

In [49]:
ws_lp, _ = lp_pmd(datasets, [0.4, 0.0, 0.4])
ws_lp_perm, _ = lp_pmd(datasets_perm, [0.4, 0.0, 0.4])

In [50]:
test_weights(ws_lp, ws_lp_perm, [1,2,0], dec=5)

True

As we can see, the weights are the same with merely the order permuted as is appropriate.

### Iterative Optimization

In [51]:
ws_io, _ = multicca_pmd(datasets, penalties)
ws_io_perm, _ = multicca_pmd(datasets_perm, penalties)

Test with decimal tolerance 5.

In [52]:
test_weights(np.array(ws_io), np.array(ws_io_perm), [1,2,0])

False

Test with decimal tolerance 2.

In [53]:
test_weights(np.array(ws_io), np.array(ws_io_perm), [1,2,0], dec=2)

False

**Note**: linear programming will always solve for the same objective function regardless of the order of the inputs, but the custom PMD implementation in `multicca_pmd` will not. Depending on the dataset, the solution may converge similarly regardless of order (with only a difference in sign), or the solution may converge to a completely different local minima given a different order.